In [52]:
import networkx as nx
import torch
import copy
import numpy as np
import scipy
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.utils as utils
from torch_geometric.datasets import Planetoid

In [138]:
class MLP(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim=None, sigma=False, bias=False):
        super(MLP, self).__init__()
        self.linear_1 = nn.Linear(input_dim, hidden_dim, bias=bias)
        if output_dim != None:
            self.linear_2 = nn.Linear(hidden_dim, output_dim, bias=bias)
        self.sigma = sigma
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
    def forward(self, x):
        x = self.linear_1(x)
        if self.sigma and self.output_dim != None:
            x = F.relu(x)
            x = self.linear_2(x)
        return (x)
    

    
class Node:
    
    def __init__(self, x, y, mlp, learning_rate, node_id):
        
        self.x = x
        self.y = y.view(1)
        self.mlp = mlp
        self.node_id = node_id
        self.learning_rate = learning_rate
        self.optimizer = torch.optim.SGD(self.mlp.parameters(), lr=self.learning_rate)  
        
    def receive_params(self, parameters):
        
        # parameters: a dict of parameters: {"linear_1.weight": tensor,......}
        with torch.no_grad():
            for name, param in self.mlp.named_parameters():
                param.copy_(parameters[name])
                
        
    def upload_params(self):
        
        # return a dict of parameters: {"linear_1,weight":" tensor,......,}
        return mlp.state_dict()
    
    # Use this funtion only at the beginning of each communication
    def upload_h(self): 
        
        self.optimizer.zero_grad()
        h = self.mlp(self.x)
        return h
    
    def receive_and_update(self, Abar_v, h_u, h_v, E):
        
        
        # h_u is the aggregated information of the neighboors of node v, it is constant
        # Abar_v is the corresponding constant for h_v and it is constant
        
        Z_v = (Abar_v*h_v + h_u)
        yhat_v = F.log_softmax(Z_v, dim=0)
        loss = F.nll_loss(yhat_v.view(1,-1), self.y)
        loss.backward()
        self.optimizer.step()
        
        for e in range(E-1):
            self.optimizer.zero_grad()
            h_v = self.mlp(self.x)
            Z_v = (Abar_v*h_v + h_u)
            yhat_v = F.log_softmax(Z_v, dim=0)
            loss = F.nll_loss(yhat_v.view(1,-1), self.y)
            loss.backward()
            self.optimizer.step()
        
        
        
        
class Central_Server:
    
    def __init__(self, edge_index, node_list, A, K, alpha=0.95):
        
        self.edge_index = edge_index
        self.node_list = node_list
        self.A = A
        self.alpha = alpha
        self.K = K
        self.Abar = None
        self.central_params = None
        self.num_nodes = len(node_list)
        
    def testing_mlp(self, input_dim, hidden_dim, output_dim=None, sigma=False):
        self.mlp = MLP(input_dim, hidden_dim, output_dim, sigma)
        
        
        
    def receive_h(self):
        
        H = []
        for v in self.node_list:
            h_v = v.upload_h()
            with torch.no_grad():
                H.append(h_v)
        return H
    
    def compute_Abar(self):
        
        N = self.A.shape[0]
        A_bar = torch.zeros((N,N))
        A_i = torch.diag(torch.ones(N))
        alpha_i = 1
        for i in range(0, self.K+1):
            A_bar = A_bar + alpha_i*A_i
            alpha_i = alpha_i * self.alpha
            A_i = torch.matmul(A_i, self.A)
        A_bar = (1-self.alpha)*A_bar
        self.Abar = A_bar
    
    def init_params(self):
        
        
        params = copy.deepcopy(self.node_list[0].mlp.state_dict())
        self.cparam_names = list(params.keys())
        
        with torch.no_grad():
            for param_name in params.keys():
                nn.init.normal_(params[param_name])
        self.central_params = params
        print (self.central_params)
        
        
            
    def one_communication(self, E):
        
        # Calculate the A_bar matrix, if it is not calculated already.
        if (self.Abar == None):
            self.compute_Abar()
            
        # If this is the first time, initilalize the central parameters.
        if (self.central_params == None):
            self.init_params()
            
        # Broadcasting the current central parameters to all nodes:
        for v in self.node_list:
            v.receive_params(self.central_params)
            
        # forwardn through the MLP layers for each node
        H = self.receive_h()
        
        # Power iteration
        H_copy = torch.stack(H).clone().detach()
        Z_K = torch.matmul(self.Abar, H_copy)
        
        # Shape of H
        num_nodes, num_classes = H_copy.shape
        
        # Calculate the needed information for each node to get the yhat
        for i in range(num_nodes):
            with torch.no_grad():
                h_u = Z_K[i,:] - H_copy[i,:]*self.Abar[i,i]
            # Local update
            self.node_list[i].receive_and_update(self.Abar[i,i], h_u, H[i], E)
            
        # Collect the updated local parameters and aggregate
        with torch.no_grad():
            for pname in self.cparam_names:
                p = self.node_list[0].mlp.state_dict()[pname]
                for i in range(1, self.num_nodes):
                    p = p + self.node_list[i].mlp.state_dict()[pname]
                p = p/self.num_nodes
                self.central_params[pname] = p
            
            
    def training(self, T, E):
        
        for t in range(T):
            self.one_communication(E)
            print ("Communication:", t+1)
            
    def training_accuracy(self, graph_data, params):
        X = graph_data.x
        Y = graph_data.y
        self.mlp.load_state_dict(params)
        with torch.no_grad():
                H = self.mlp(X)
                Z_K = torch.matmul(self.Abar, H)
                preds = torch.max(Z_K, dim=1)[1]
                counts = (preds == Y).sum()
        print (counts)

In [139]:
def init_network(data, A, num_classes, hidden_dim, learning_rate, K,
                       output_dim=None, sigma=False):
    
    num_nodes, input_dim = data.x.shape
    node_list = []
    
    for v in range(num_nodes):
        
        if (output_dim != None and sigma == True):
            mlp = MLP(input_dim, hidden_dim, output_dim, sigma)
            
        else:
            mlp = MLP(input_dim, hidden_dim)
            
        node_v = Node(data.x[v,:], data.y[v], mlp, learning_rate, v)
        node_list.append(node_v)
    
    network = Central_Server(data.edge_index, node_list, A, K)
    
    network.testing_mlp(input_dim, hidden_dim, output_dim, sigma)
    
    return network

In [149]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
data.edge_index = utils.remove_self_loops(data.edge_index)[0]
data.edge_index = utils.add_remaining_self_loops(data.edge_index)[0]
G = utils.to_networkx(data, to_undirected=True)
A = torch.tensor(nx.linalg.graphmatrix.adjacency_matrix(G).todense()).type(torch.FloatTensor)
D = nx.linalg.graphmatrix.adjacency_matrix(G).todense() + nx.linalg.laplacianmatrix.laplacian_matrix(G).todense()
Dhalf = scipy.linalg.sqrtm(D)
Dnhalf = torch.tensor(scipy.linalg.inv(Dhalf)).type(torch.FloatTensor)
A = torch.matmul(torch.matmul(Dnhalf, A), Dnhalf)
server = init_network(data, A, 7, 7, 0.1, 20)
server.training(20, 1)

OrderedDict([('linear_1.weight', tensor([[ 0.2780,  0.4091, -0.8773,  ..., -0.0978,  1.0145, -0.5874],
        [ 0.5348,  0.5002, -0.6751,  ..., -0.2304,  1.0212,  0.6547],
        [-1.9875, -0.7744,  2.8199,  ..., -0.5654,  1.0590,  0.6054],
        ...,
        [-0.3207,  1.3769,  0.4110,  ...,  1.2728,  0.2249, -0.9476],
        [ 0.4499,  0.9240, -0.6139,  ...,  0.4759,  1.2984,  0.9053],
        [ 0.7185, -0.7133,  0.0995,  ...,  0.7702, -0.0281,  1.3712]]))])
Communication: 1
Communication: 2
Communication: 3
Communication: 4
Communication: 5
Communication: 6
Communication: 7
Communication: 8
Communication: 9
Communication: 10
Communication: 11
Communication: 12
Communication: 13
Communication: 14
Communication: 15
Communication: 16
Communication: 17
Communication: 18
Communication: 19
Communication: 20


In [150]:
server.central_params

OrderedDict([('linear_1.weight',
              tensor([[ 0.2777,  0.4087, -0.8773,  ..., -0.0976,  1.0144, -0.5874],
                      [ 0.5348,  0.5002, -0.6750,  ..., -0.2304,  1.0213,  0.6547],
                      [-1.9874, -0.7744,  2.8198,  ..., -0.5654,  1.0588,  0.6053],
                      ...,
                      [-0.3207,  1.3769,  0.4106,  ...,  1.2727,  0.2244, -0.9475],
                      [ 0.4499,  0.9241, -0.6139,  ...,  0.4759,  1.2984,  0.9053],
                      [ 0.7185, -0.7134,  0.0986,  ...,  0.7701, -0.0283,  1.3711]]))])